In [1]:
# Import packages
import ee
import geemap
import os
import geopandas as gpd

ee.Initialize() # Initialize

In [29]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [30]:
Map.user_roi # Shows you the polygon
Map.user_roi.getInfo() # More specific
roi = Map.user_roi # Set region of interest

In [31]:
# This collection includes daily images on Soil Moisture ranging from 2000-02-24 up to now.
collection = ee.ImageCollection("MODIS/061/MOD13Q1") # Load collection for NDVI
print(collection.size().getInfo()) # Shows the number of images within the collectionz

505


In [32]:
collection_subset = ee.ImageCollection("MODIS/061/MOD13Q1") \
    .filterDate('2012-01-01', '2022-01-01').sort('IMAGE_DATE').select('NDVI') # Only select images for the years 2016-2019
print(collection_subset.size().getInfo()) # Shows the number of images within the subcollection
image = collection_subset.first().select('NDVI')  # Pick the first image from the 'list' and select the layer of interest
geemap.image_props(image).getInfo() # Finds basic info of this image

230


{'IMAGE_DATE': '2012-01-01',
 'NOMINAL_SCALE': 231.65635826395828,
 'google:max_source_file_timestamp': 1629143093000,
 'system:asset_size': '29545.941895 MB',
 'system:band_names': ['NDVI'],
 'system:id': 'MODIS/061/MOD13Q1/2012_01_01',
 'system:index': '2012_01_01',
 'system:time_end': '2012-01-17 00:00:00',
 'system:time_start': '2012-01-01 00:00:00',
 'system:version': 1646291864802594}

In [34]:
import os
out  = str(os.path.abspath(os.getcwd()))+'/NDVI/Maharashtra'
out

'/home/jovyan/Untitled Folder/NDVI/Maharashtra'

In [35]:
# Now export each image within the collection to a GeoTIF
geemap.ee_export_image_collection(collection_subset, out_dir = out, scale=image.select('NDVI').projection().nominalScale(), region=roi, file_per_band=True, crs='EPSG:4326')#, maxPixels = 1e13)

# image.select('ssm').projection().nominalScale() sets the scale equal to the resolution of the images within the collection
# file_per_band=False: all bands are downloaded and put as one file
# file_per_band=True: each band is downloaded in a single image

Total number of images: 230

Exporting 1/230: 2012_01_01.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 2/230: 2012_01_17.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 3/230: 2012_02_02.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 4/230: 2012_02_18.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 5/230: 2012_03_05.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 6/230: 2012_03_21.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 7/230: 2012_04_06.tif
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/Untitled Folder/NDVI/Maharashtra


Exporting 8/230: 2012_04_22.tif
G